# 範例 : 計程車費率預測
https://www.kaggle.com/c/new-york-city-taxi-fare-prediction

# [作業目標]
- 試著模仿範例寫法, 使用程車費率預測競賽練習時間欄位處理

# [作業重點]
- 新增星期幾(day of week)與第幾周(week of year)這兩項特徵, 觀察有什麼影響 (In[4], Out[4], In[5], Out[5])
- 新增加上年週期與周周期特徵 , 觀察有什麼影響 (In[8], Out[8], In[9], Out[9]) 

In [15]:
# 程式區塊 A
# 將需要的都import進來
import os
import copy
import time
import math
import numpy             as np
import pandas            as pd
import seaborn           as sns
import datetime          as dt
import warnings
import matplotlib.pyplot as plt
from scipy                   import stats
from sklearn.ensemble        import GradientBoostingRegressor
from sklearn.linear_model    import LogisticRegression,LinearRegression
from sklearn.preprocessing   import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score

# 將較長的函式改名一下
MME  = MinMaxScaler()
LE   = LabelEncoder()
LR   = LogisticRegression()
LIR  = LinearRegression()
GBR  = GradientBoostingRegressor()
PDDF = pd.DataFrame()
# 一些必要的設定
warnings.filterwarnings('ignore')
%matplotlib inline

# 設定【data的資料夾路徑】，命名為【data_folder】
data_folder = 'C:/Users/Ynitsed/Documents/GitHub/2nd-ML100Days/data'

In [16]:
# 設定t001為某個data路徑
# 設定t002為pd裡read data的功能
t001_train = os.path.join(data_folder, 'taxi_data1.csv')
t002_train = pd.read_csv(t001_train)
print('Path of read in data: %s' %t001_train)
print(t002_train.shape)
t002_train.head()

Path of read in data: C:/Users/Ynitsed/Documents/GitHub/2nd-ML100Days/data\taxi_data1.csv
(5000, 7)


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2011-10-21 23:54:10 UTC,-73.990580,40.761071,-73.981128,40.758634,2
1,12.0,2015-02-03 10:42:03 UTC,-73.988403,40.723431,-73.989647,40.741695,1
2,6.5,2014-03-16 18:58:58 UTC,-74.015785,40.715110,-74.012029,40.707888,2
3,6.5,2009-06-13 16:10:54 UTC,-73.977322,40.787275,-73.958030,40.778838,3
4,11.0,2014-06-12 03:25:56 UTC,-73.989683,40.729717,-73.982490,40.761887,3


In [17]:
# 把【train_Y】先列出來
train_Y = t002_train['fare_amount']
# 把【train_Y】要的欄位拿掉，可以當作在處理【train_X】，只是還沒到最終步。
t003_train = t002_train.drop(['fare_amount'] , axis=1)
t003_train.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2011-10-21 23:54:10 UTC,-73.990580,40.761071,-73.981128,40.758634,2
1,2015-02-03 10:42:03 UTC,-73.988403,40.723431,-73.989647,40.741695,1
2,2014-03-16 18:58:58 UTC,-74.015785,40.715110,-74.012029,40.707888,2
3,2009-06-13 16:10:54 UTC,-73.977322,40.787275,-73.958030,40.778838,3
4,2014-06-12 03:25:56 UTC,-73.989683,40.729717,-73.982490,40.761887,3


In [18]:
# 時間特徵分解方式:使用datetime
t003_train['pickup_datetime'] = t003_train['pickup_datetime'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S UTC'))
t003_train['pickup_year']     = t003_train['pickup_datetime'].apply(lambda x: dt.datetime.strftime(x, '%Y')).astype('int64')
t003_train['pickup_month']    = t003_train['pickup_datetime'].apply(lambda x: dt.datetime.strftime(x, '%m')).astype('int64')
t003_train['pickup_day']      = t003_train['pickup_datetime'].apply(lambda x: dt.datetime.strftime(x, '%d')).astype('int64')
t003_train['pickup_hour']     = t003_train['pickup_datetime'].apply(lambda x: dt.datetime.strftime(x, '%H')).astype('int64')
t003_train['pickup_minute']   = t003_train['pickup_datetime'].apply(lambda x: dt.datetime.strftime(x, '%M')).astype('int64')
t003_train['pickup_second']   = t003_train['pickup_datetime'].apply(lambda x: dt.datetime.strftime(x, '%S')).astype('int64')
t003_train.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_second
0,2011-10-21 23:54:10,-73.990580,40.761071,-73.981128,40.758634,2,2011,10,21,23,54,10
1,2015-02-03 10:42:03,-73.988403,40.723431,-73.989647,40.741695,1,2015,2,3,10,42,3
2,2014-03-16 18:58:58,-74.015785,40.715110,-74.012029,40.707888,2,2014,3,16,18,58,58
3,2009-06-13 16:10:54,-73.977322,40.787275,-73.958030,40.778838,3,2009,6,13,16,10,54
4,2014-06-12 03:25:56,-73.989683,40.729717,-73.982490,40.761887,3,2014,6,12,3,25,56


In [19]:
# 將結果使用線性迴歸 / 梯度提升樹分別看結果
t004_train = t003_train.drop(['pickup_datetime'] , axis=1)
train_X = MME.fit_transform(t004_train)
print(f'Linear Reg Score : {cross_val_score(LIR, train_X, train_Y, cv=5).mean()}')
print(f'Gradient Boosting Reg Score : {cross_val_score(GBR, train_X, train_Y, cv=5).mean()}')

Linear Reg Score : 0.026876871475636888
Gradient Boosting Reg Score : 0.7114221612981696


In [3]:
# 將結果使用線性迴歸 / 梯度提升樹分別看結果
df_temp = df.drop(['pickup_datetime'] , axis=1)
scaler = MinMaxScaler()
train_X = scaler.fit_transform(df_temp)
Linear = LinearRegression()
print(f'Linear Reg Score : {cross_val_score(Linear, train_X, train_Y, cv=5).mean()}')
GDBT = GradientBoostingRegressor()
print(f'Gradient Boosting Reg Score : {cross_val_score(GDBT, train_X, train_Y, cv=5).mean()}')

Linear Reg Score : 0.026876871475640173
Gradient Boosting Reg Score : 0.7115599753640718


# 作業1
* 對照範例，試著加入星期幾 (day of week) 與第幾周 (week of year) 這兩項特徵，  
看看結果會比原本只有時間特徵分解的結果更好或更差?

In [20]:
t003_train['pickup_dow'] = t003_train['pickup_datetime'].apply(lambda x: dt.datetime.strftime(x, '%w')).astype('int64')
t003_train['pickup_woy'] = t003_train['pickup_datetime'].apply(lambda x: dt.datetime.strftime(x, '%W')).astype('int64')
t003_train.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_second,pickup_dow,pickup_woy
0,2011-10-21 23:54:10,-73.990580,40.761071,-73.981128,40.758634,2,2011,10,21,23,54,10,5,42
1,2015-02-03 10:42:03,-73.988403,40.723431,-73.989647,40.741695,1,2015,2,3,10,42,3,2,5
2,2014-03-16 18:58:58,-74.015785,40.715110,-74.012029,40.707888,2,2014,3,16,18,58,58,0,10
3,2009-06-13 16:10:54,-73.977322,40.787275,-73.958030,40.778838,3,2009,6,13,16,10,54,6,23
4,2014-06-12 03:25:56,-73.989683,40.729717,-73.982490,40.761887,3,2014,6,12,3,25,56,4,23


In [21]:
# 將結果使用線性迴歸 / 梯度提升樹分別看結果
t005_train = t003_train.drop(['pickup_datetime'] , axis=1)
train_X = MME.fit_transform(t005_train)
print(f'Linear Reg Score : {cross_val_score(LIR, train_X, train_Y, cv=5).mean()}')
print(f'Gradient Boosting Reg Score : {cross_val_score(GBR, train_X, train_Y, cv=5).mean()}')

Linear Reg Score : 0.02864900696250865
Gradient Boosting Reg Score : 0.7106906266549817


In [22]:
# 加上"日週期"特徵 (參考講義"週期循環特徵")
t003_train['day_cycle'] = t003_train['pickup_hour']/12 + t003_train['pickup_minute']/720 + t003_train['pickup_second']/43200
t003_train['day_cycle'] = t003_train['day_cycle'].map(lambda x:math.sin(x*math.pi))
t003_train.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_second,pickup_dow,pickup_woy,day_cycle
0,2011-10-21 23:54:10,-73.990580,40.761071,-73.981128,40.758634,2,2011,10,21,23,54,10,5,42,-0.025450
1,2015-02-03 10:42:03,-73.988403,40.723431,-73.989647,40.741695,1,2015,2,3,10,42,3,2,5,0.333601
2,2014-03-16 18:58:58,-74.015785,40.715110,-74.012029,40.707888,2,2014,3,16,18,58,58,0,10,-0.967083
3,2009-06-13 16:10:54,-73.977322,40.787275,-73.958030,40.778838,3,2009,6,13,16,10,54,6,23,-0.888817
4,2014-06-12 03:25:56,-73.989683,40.729717,-73.982490,40.761887,3,2014,6,12,3,25,56,4,23,0.782427


In [23]:
# 將結果使用線性迴歸 / 梯度提升樹分別看結果
t006_train = t003_train.drop(['pickup_datetime'] , axis=1)
train_X = MME.fit_transform(t006_train)
print(f'Linear Reg Score : {cross_val_score(LIR, train_X, train_Y, cv=5).mean()}')
print(f'Gradient Boosting Reg Score : {cross_val_score(GBR, train_X, train_Y, cv=5).mean()}')

Linear Reg Score : 0.02827400966881628
Gradient Boosting Reg Score : 0.7120093428567287


# 作業2
* 對照範例的日週期效果，試著參考投影片完成年週期與周週期的特徵 (也可以用你自己想到的方式)，  
看看結果會比範例中的結果更好或更差?

In [24]:
# 加上"年週期"與"周週期"特徵
t003_train['year_cycle'] = t003_train['pickup_month']/6 + t003_train['pickup_day']/180
t003_train['year_cycle'] = t003_train['year_cycle'].map(lambda x:math.cos(x*math.pi))
t003_train['week_cycle'] = t003_train['pickup_dow']/3.5 + t003_train['pickup_hour']/84
t003_train['week_cycle'] = t003_train['week_cycle'].map(lambda x:math.sin(x*math.pi))
t003_train.head()

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_second,pickup_dow,pickup_woy,day_cycle,year_cycle,week_cycle
0,2011-10-21 23:54:10,-73.990580,40.761071,-73.981128,40.758634,2,2011,10,21,23,54,10,5,42,-0.025450,0.777146,-0.804598
1,2015-02-03 10:42:03,-73.988403,40.723431,-73.989647,40.741695,1,2015,2,3,10,42,3,2,5,0.333601,0.453990,0.826239
2,2014-03-16 18:58:58,-74.015785,40.715110,-74.012029,40.707888,2,2014,3,16,18,58,58,0,10,-0.967083,-0.275637,0.623490
3,2009-06-13 16:10:54,-73.977322,40.787275,-73.958030,40.778838,3,2009,6,13,16,10,54,6,23,-0.888817,-0.974370,-0.294755
4,2014-06-12 03:25:56,-73.989683,40.729717,-73.982490,40.761887,3,2014,6,12,3,25,56,4,23,0.782427,-0.978148,-0.532032


In [25]:
# 將結果使用線性迴歸 / 梯度提升樹分別看結果
t007_train = t003_train.drop(['pickup_datetime'] , axis=1)
train_X = MME.fit_transform(t007_train)
print(f'Linear Reg Score : {cross_val_score(LIR, train_X, train_Y, cv=5).mean()}')
print(f'Gradient Boosting Reg Score : {cross_val_score(GBR, train_X, train_Y, cv=5).mean()}')

Linear Reg Score : 0.028326599866688795
Gradient Boosting Reg Score : 0.7138261092745917


### Day25教材方向和目標
對於時間的處理語法

### Day25忽略部分
datatime其他細節

### Day25其他補充
我覺得這分數都沒啥差異，答案說【正確性有顯著的提升】真不知道是怎麼想的。